In [1]:
import pandas as pd  # For data manipulation
import glob  # For file handling
from pyproj import Transformer 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

ModuleNotFoundError: No module named 'pyproj'

In [305]:
folder_path = '/Users/sofianebelbrik/Documents/water pollution/Data/Raw/'

In [307]:
years = range(2020, 2025)

In [ ]:
# Creating an empty list to store file paths
files = []

# Looping through each year and find the corresponding CSV files
for year in years:
    file_pattern = folder_path + f'{year}-M.csv'  # Format file name pattern
    files.extend(glob.glob(file_pattern))  # Append all matching files to the list

# Printing the files that were found
print("Files found:", files)

# Loading all CSV files into a list of DataFrames
df_list = [pd.read_csv(file) for file in files]  # Read each file into a DataFrame

# Concatenating all DataFrames into one large DataFrame
data_2020_2025 = pd.concat(df_list, ignore_index=True)

# Displaying the shape of the final dataset (for debugging)
print("Final dataset shape:", data_2020_2025.shape)  



In [ ]:
df_list = [pd.read_csv(file) for file in files]
data_2020_2024 = pd.concat(df_list, ignore_index=True)
print(data_2020_2024.shape) 

In [ ]:
data_2020_2024.head()

In [ ]:
# Defining columns to retain
columns_to_keep = [
    'sample.sampleDateTime',   # Timestamp of the sample collection
    'determinand.label',       # Water quality parameter name
    'result',                  # Measurement value of the parameter
    'determinand.unit.label',  # Unit of measurement
    'sample.samplingPoint.easting',  # X coordinate of the sampling location
    'sample.samplingPoint.northing', # Y coordinate of the sampling location
    'sample.sampledMaterialType.label'  # Type of water 
]

# Keeping only these columns in the dataset
data_2020_2024 = data_2020_2024[columns_to_keep]

# Displaying the first few rows for verification
print("Preview of filtered dataset:")
print(data_2020_2024.head())

In [ ]:
# Listing unique values in 'determinand.label' column 
unique_determinand_labels = data_2020_2024['determinand.label'].unique()

# Displaying unique parameter names
print("Unique water quality parameters in dataset:")
print(unique_determinand_labels)

In [ ]:
# Defining the key water quality parameters to keep
key_parameters = [
    'O Diss %sat',   # Oxygen Dissolved Percentage Saturation
    'Orthophospht',   # Orthophosphate
    'Ammonia(N)',     # Ammonia Nitrogen
    'Temp Water',     # Water Temperature
    'pH',             # pH Level
    'Nitrite-N',      # Nitrite Nitrogen
    'Nitrate-N',      # Nitrate Nitrogen
    'TurbidityNTU',   # Turbidity
    'BOD ATU',        # Biochemical Oxygen Demand
    'Phosphorus-P'    # Phosphorus
]

# Filtering the dataset to keep only these parameters
data_2020_2024 = data_2020_2024[data_2020_2024['determinand.label'].isin(key_parameters)]

# Confirming the filtering
print("Dataset shape after filtering key water parameters:", data_2020_2024.shape)

In [ ]:
# Listing unique water types present in the dataset
unique_water_types = data_2020_2024['sample.sampledMaterialType.label'].unique()

# Displaying unique water types
print("Unique water types in dataset:")
print(unique_water_types)

In [ ]:
# Defining relevant water types for drinking and bathing
key_water_types = [
    'GROUNDWATER',  # Underground water (important for drinking)
    'RIVER / RUNNING SURFACE WATER',  # Flowing water bodies
    'POND / LAKE / RESERVOIR WATER'  # Still water bodies
]

# Filtering dataset to keep only these water types
data_2020_2024 = data_2020_2024[data_2020_2024['sample.sampledMaterialType.label'].isin(key_water_types)]

# Confirming the filtering
print("Dataset shape after filtering key water types:", data_2020_2024.shape)

In [ ]:
print("Number of unique Easting values:", data_2020_2024['sample.samplingPoint.easting'].nunique())
print("Number of unique Northing values:", data_2020_2024['sample.samplingPoint.northing'].nunique())

In [ ]:
data_2020_2024.head(40)

In [ ]:
# Finding unique measurement units for each parameter
units_per_parameter = data_2020_2024.groupby("determinand.label", observed=False)["determinand.unit.label"].unique()

# Converting to a DataFrame for better readability
units_data_2020_2024 = units_per_parameter.explode().reset_index()

# Displaying the units for each parameter
print("Measurement units for each water quality parameter:")
print(units_data_2020_2024)

In [ ]:
num_unique_locations = data_2020_2024[['sample.samplingPoint.easting', 'sample.samplingPoint.northing']].drop_duplicates().shape[0]
print(f"Number of Unique Locations: {num_unique_locations}")

In [ ]:
# Converting the 'sample.sampleDateTime' column to datetime format for time-based analysis
data_2020_2024['sample.sampleDateTime'] = pd.to_datetime(data_2020_2024['sample.sampleDateTime'], errors='coerce')

# Verifying conversion by checking the first few values
print("DateTime conversion preview:")
print(data_2020_2024['sample.sampleDateTime'].head())

In [ ]:
data_2020_2024.isnull().sum()

In [ ]:
# Removing duplicate rows to prevent redundant data
data_2020_2024.drop_duplicates(inplace=True)

# Confirming the number of unique rows after dropping duplicates
print("Dataset shape after removing duplicates:", data_2020_2024.shape)

In [ ]:
# Dictionary to map inconsistent unit names to standard ones
unit_conversion = {
    "cel": "°C",       # Convert temperature unit to Celsius symbol
    "mg/l": "mg/L",    # Standardize milligrams per liter (mg/L)
    "ntu": "NTU",      # Standardize Turbidity unit
    "phunits": "pH"    # Simplify pH unit name
}

# Applying unit conversion mapping to the 'determinand.unit.label' column
data_2020_2024["determinand.unit.label"] = data_2020_2024["determinand.unit.label"].replace(unit_conversion)

# Displaying unique unit values after standardization
print("Unique units after standardization:", data_2020_2024["determinand.unit.label"].unique())

In [ ]:
# Pivoting the data to restructure it: each parameter becomes a separate column
data_pivot = data_2020_2024.pivot_table(
    index=["sample.sampleDateTime", "sample.samplingPoint.easting", "sample.samplingPoint.northing"],  
    columns="determinand.label",  
    values="result"
).reset_index()

# Renaming columns for better clarity
data_pivot.columns.name = None  # Remove the MultiIndex column name

# Displaying the first few rows
print("Pivoted dataset preview:")
print(data_pivot.head())

In [ ]:
data_pivot.head

In [ ]:
data_pivot.head()

In [ ]:
# Calculating the percentage of missing values in each column
missing_percent = (data_pivot.isnull().sum() / len(data_pivot)) * 100

# Print missing values percentage
print("Missing Values Percentage:\n", missing_percent)

In [244]:
# Sorting the dataset by date to ensure proper filling
data_pivot.sort_values(by=["sample.sampleDateTime"], inplace=True)

In [246]:
# Forwarding fill: fills missing values using the last known value
data_pivot.ffill(inplace=True)

# Backwarding fill: fills missing values using the next known value (if any remain)
data_pivot.bfill(inplace=True)

In [ ]:
data_2020_2024

In [ ]:
# Confirm that all missing values have been handled
print("Missing values after filling:\n", data_pivot.isnull().sum())  # Should print all 0s

In [ ]:
data_pivot.head()

In [ ]:
# Check the number of unique locations in the data based on Easting and Northing
unique_locations = data_pivot[['sample.samplingPoint.easting', 'sample.samplingPoint.northing']]
num_unique_locations = unique_locations.shape[0]

num_unique_locations

In [ ]:
# Count unique locations in data_pivot
num_unique_locations_pivot = data_pivot[['sample.samplingPoint.easting', 'sample.samplingPoint.northing']].drop_duplicates().shape[0]
print(f"Unique locations in data_pivot: {num_unique_locations_pivot}")

In [ ]:

# Definin the outlier detection function using the IQR method
def detect_outliers_iqr(df, columns):
    """
    Detect outliers in the given DataFrame 'df' for the specified 'columns'
    using the Interquartile Range (IQR) method.
    """
    outliers = {}
    for col in columns:
        Q1 = df[col].quantile(0.25)  # 25th percentile
        Q3 = df[col].quantile(0.75)  # 75th percentile
        IQR = Q3 - Q1                # Interquartile Range

        # Define lower and upper bounds for outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Count outliers in the column
        outlier_count = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
        outliers[col] = {
            "Outlier Count": outlier_count,
            "Lower Bound": lower_bound,
            "Upper Bound": upper_bound
        }
    
    # Converting the dictionary to a DataFrame for easier readability
    return pd.DataFrame.from_dict(outliers, orient="index")

# Get a list of numeric columns (excluding time and location columns)
numeric_columns = [
    col for col in data_pivot.columns 
    if col not in ["sample.sampleDateTime", "sample.samplingPoint.easting", "sample.samplingPoint.northing"]
]

# Detect the outliers on the pivoted DataFrame
outlier_df = detect_outliers_iqr(data_pivot, numeric_columns)

# Displaying the outlier detection results
print("Outlier Detection Results:")
print(outlier_df)

In [ ]:
# Creating a list of numeric columns, excluding datetime and location columns.
numeric_columns = [
    col for col in data_pivot.columns 
    if col not in ["sample.sampleDateTime", "sample.samplingPoint.easting", "sample.samplingPoint.northing"]
]

# Printing summary statistics BEFORE winsorization.
print("Summary Statistics BEFORE Winsorization:")
print(data_pivot[numeric_columns].describe())

# Defining the winsorization function.
def winsorize_series(series, factor=1.5):
    """
    Winsorize a Pandas Series using the Interquartile Range (IQR) method.
    
    Parameters:
        series (pd.Series): The data series to winsorize.
        factor (float): The multiplier for the IQR to determine bounds (default is 1.5).
    
    Returns:
        pd.Series: The winsorized series where values below the lower bound are set to the lower bound
                   and values above the upper bound are set to the upper bound.
    """
    # Calculating the first and third quartiles.
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    
    # Calculating the Interquartile Range (IQR).
    IQR = Q3 - Q1
    
    # Defining the lower and upper bounds based on the factor.
    lower_bound = Q1 - factor * IQR
    upper_bound = Q3 + factor * IQR
    
    # Cliping the series so that all values fall within the lower and upper bounds.
    return series.clip(lower=lower_bound, upper=upper_bound)

# Applying winsorization to each numeric column in data_pivot.
for col in numeric_columns:
    data_pivot[col] = winsorize_series(data_pivot[col])

# Printing summary statistics AFTER winsorization to see how the distribution has changed.
print("\nSummary Statistics AFTER Winsorization:")
print(data_pivot[numeric_columns].describe())

In [ ]:
plt.figure(figsize=(12, 6))
data_pivot[numeric_columns].boxplot(rot=90)
plt.title("Boxplot of Water Quality Parameters After Winsorization")
plt.show()

In [256]:
# Extracting the year from the sample date/time
data_pivot['year'] = data_pivot['sample.sampleDateTime'].dt.year

# Extracting the month from the sample date/time
data_pivot['month'] = data_pivot['sample.sampleDateTime'].dt.month

# Extracting the day of the month from the sample date/time
data_pivot['day'] = data_pivot['sample.sampleDateTime'].dt.day

# Extracting the day of the week (0=Monday, 6=Sunday) from the sample date/time
data_pivot['day_of_week'] = data_pivot['sample.sampleDateTime'].dt.dayofweek

In [ ]:
# Extracting spatial features (Easting and Northing) from the DataFrame.
spatial_features = data_pivot[['sample.samplingPoint.easting', 'sample.samplingPoint.northing']]

# Initialising KMeans with 10 clusters.
# The 'random_state=42' parameter ensures that the clustering is reproducible.
kmeans = KMeans(n_clusters=10, random_state=42)

# Fitting the KMeans model on the spatial features and assign each row a cluster label.

data_pivot['location_cluster'] = kmeans.fit_predict(spatial_features)

print("Cluster Centers (Easting, Northing):")
print(kmeans.cluster_centers_)

print("Samples per Cluster:")
print(data_pivot['location_cluster'].value_counts())

In [ ]:
data_pivot.head()


In [ ]:
"""
This script processes the 'data_pivot' DataFrame by:
1. Sorting the data by spatial coordinates and time.
2. Computing a 7-observation rolling average for each numeric column,
   grouped by unique spatial locations (defined by easting and northing).
"""
data_pivot = data_pivot.sort_values(
    by=['sample.samplingPoint.easting', 'sample.samplingPoint.northing', 'sample.sampleDateTime']
)

# Looping through each numeric column
# and compute a 7-observation rolling average for each unique location.
for col in numeric_columns:
    # Creating a new column name for the rolling average (e.g., "pH_rolling7" for a column named "pH")
    new_col = f"{col}_rolling7"
    
    # Grouping by spatial location and calculate the rolling mean with a window of 7 observations.
    # The 'min_periods=1' parameter ensures that even if there are fewer than 7 observations,
    # the mean is computed from the available data.
    data_pivot[new_col] = (
        data_pivot.groupby(['sample.samplingPoint.easting', 'sample.samplingPoint.northing'])[col]
                  .rolling(window=7, min_periods=1)
                  .mean()
                  .reset_index(level=[0, 1], drop=True)
    )


print("Data sample with rolling averages:")
print(data_pivot.head())


In [ ]:
data_pivot.head()

In [ ]:

data_pivot.head()

In [ ]:
"""
Lag Feature Engineering for Water Quality Data
------------------------------------------------
This script creates lag features for numeric water quality parameters
from the 'data_pivot' DataFrame. For each unique location, it calculates:
    - A 1-step lag (previous observation)
    - A 2-step lag (observation before the previous one)
After creating the lag features, missing values are handled via forward-fill
followed by backward-fill within each location group.
"""

 # Create a copy of data_pivot for lag feature engineering
data_lag = data_pivot.copy()

# Looping through each numeric column and create lag features for each unique location.
for col in numeric_columns:
    # Create a new column for the 1-step lag (previous observation)
    lag1_col = f"{col}_lag1"
    data_lag[lag1_col] = data_lag.groupby(
        ['sample.samplingPoint.easting', 'sample.samplingPoint.northing']
    )[col].shift(1)
    
    # Creating a new column for the 2-step lag (the observation before the previous one)
    lag2_col = f"{col}_lag2"
    data_lag[lag2_col] = data_lag.groupby(
        ['sample.samplingPoint.easting', 'sample.samplingPoint.northing']
    )[col].shift(2)

# Handli g missing values in the lag features by applying forward-fill and backward-fill.

data_lag = data_lag.groupby(
    ['sample.samplingPoint.easting', 'sample.samplingPoint.northing']
).apply(lambda group: group.ffill().bfill()).reset_index(drop=True)


print("Data sample with lag features and missing values handled:")
print(data_lag.head())

In [ ]:
data_lag.head()

In [ ]:
# Check for missing values per column in data_pivot
print("Missing values per column:")
print(data_lag.isnull().sum())

In [193]:
data_final = data_lag.copy()

In [212]:
data_final_ = data_final.sort_values(by='sample.sampleDateTime')

In [221]:
data_final_.to_csv('final_water_quality_df.csv', index=False)